##  Sentinel-3 SLSTR composites

### Service Definition

In [1]:
service = dict([('title', 'Sentinel-3 SLSTR composites'),
                ('abstract', 'Sentinel-3 SLSTR composites for descending acquisitions'),
                ('id', 'ewf-s3-slstr-composites')])

### Parameter Definition 

In [2]:
false_color_infrared = dict([('id', 'false_color_infrared'),
                       ('title', 'False Color Infrared (S3, S2, S1)'),
                       ('abstract', 'False Color Infrared (S3, S2, S1)'),
                       ('value', 'Yes'),
                       ('options', 'Yes,No')])

In [4]:
false_color_1 = dict([('id', 'false_color_1'),
                       ('title', 'False Color 1 (S5, S3, S2)'),
                       ('abstract', 'False Color 1 (S5, S3, S2)'),
                       ('value', 'Yes'),
                       ('options', 'Yes,No')])

In [3]:
#false_color_2 = dict([('id', 'false_color_2'),
#                       ('title', 'False Color 2 (S8, S1, S5)'),
#                       ('abstract', 'False Color 2 (S8, S1, S5)'),
#                       ('value', 'Yes'),
#                       ('options', 'Yes,No')])

### Runtime parameter definition

**Input reference**

The input identifier is the catalogue entry URL (a.k.a. self value).

In [5]:
input_reference = dict([('identifier', 'input_reference'),
                        ('title', 'Sentinel-3 SLSTR Level-1 (SL_1_RBT___ descending pass)'),
                        ('abstract', 'This service takes as input a Sentinel-3 SLSTR Level 1 (SL_1_RBT___) product on DESCENDING pass and does False Colour RGB composites'),
                        ('value', 'https://catalog.terradue.com/sentinel3/search?uid=S3A_SL_1_RBT____20200530T052341_20200530T052641_20200530T072154_0180_059_005_2520_LN2_O_NR_004'),
                        ('stac:collection', 'input_reference'),
                        ('stac:href', 'catalog.json'),
                        ('max_occurs', '16')])

**Data path**

This path defines where the data is staged-in. 

In [6]:
data_path = '/workspace/data/slstr'

In [7]:
input_catalog = '/workspace/data/slstr/catalog.json'

### Workflow

#### Import the packages

In [8]:
import os
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'
import sys
sys.path.append(os.path.join(os.environ['PREFIX'], 'conda-otb/lib/python'))
os.environ['OTB_APPLICATION_PATH'] = os.path.join(os.environ['PREFIX'], 'conda-otb/lib/otb/applications')
os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
import otbApplication
import gdal
from helpers import *
from shapely.wkt import loads
from shapely.geometry import box
from shapely.geometry import shape
import shutil
from pystac import Catalog, Collection, Item, MediaType, Asset, CatalogType

gdal.UseExceptions()

In [9]:
%load_ext autoreload
%autoreload 2

In [10]:
cat = Catalog.from_file(input_catalog)

if cat is None:
    raise ValueError()

In [11]:
collection = next(cat.get_children())

In [12]:
item = next(collection.get_items())

In [15]:
item.properties['eop:orbitDirection']

'DESCENDING'

In [16]:
if item.properties['eop:orbitDirection'] != 'DESCENDING':
    ciop.log('ERROR','Product cannot be used as input')
    raise Exception('Only use products with Orbit=Descending')

### Import Sentinel-3 SLSTR product

In [17]:
operators = ['Read', 
             'Rad2Refl',
             'Resample',
             'Reproject',
             'Write']

In [26]:
read = dict()

s3_path = item.assets['metadata'].get_absolute_href()

read['file'] =  s3_path
read['formatName'] = 'Sen3_SLSTRL1B_500m'

rad2refl = dict()

rad2refl['sensor'] = 'SLSTR_500m'
rad2refl['copyTiePointGrids'] = 'true'
rad2refl['copyFlagBandsAndMasks'] = 'true'
rad2refl['copyNonSpectralBands'] = 'true'

resample = dict()
resample['referenceBandName'] = 'S1_reflectance_an'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

write = dict()
write['file'] = 's3_slstr'

In [31]:
snap_graph(os.environ['GPT_BIN'],
           operators,
           Read=read, 
           Rad2Refl=rad2refl,
           Resample=resample,
           Reproject=reproject,
           Write=write)

0

### RGB Composites

In [32]:
date_format = '%Y%m%dT%H%m%S'

output_startdate = item.datetime
output_stopdate = item.datetime

In [33]:
composites = dict()

composites['S3 SLSTR False color Infrared'] = {'bands': 'S3_reflectance_an,S2_reflectance_an,S1_reflectance_an',
                                               'create': True if (false_color_infrared['value'] == 'Yes') else False,
                                               'hfact': 3.0}

composites['S3 SLSTR False color 1'] = {'bands': 'S5_reflectance_an,S3_reflectance_an,S2_reflectance_an',
                                               'create': True if (false_color_2['value'] == 'Yes') else False,
                                               'hfact': 3.0}

#composites['S3 SLSTR False color 1'] = {'bands': 'S8_BT_in,S1_reflectance_an,S5_reflectance_an',
#                                               'create': True if (false_color_1['value'] == 'Yes') else False,
#                                               'hfact': 3.0}



In [34]:
for k, v in composites.items():
    
    if not v['create']:
        
        continue
    
    bands = [os.path.join(write['file'] + '.data',  '{}.img'.format(band)) for band in (composites[k]['bands'].split(',') +  
                                                                         ['cloud_an',
                                                                          'confidence_an',
                                                                          'S2_exception_an'])]
    
    print(bands)
    
    ds = gdal.Open(bands[0])

    geo_transform = ds.GetGeoTransform()
    projection_ref = ds.GetProjectionRef()
    
    ds = None
    
    s3_rgb_data = read_s3(bands)
    
    red = s3_rgb_data[:,:,0]
    green = s3_rgb_data[:,:,1]
    blue = s3_rgb_data[:,:,2]
    cloud = s3_rgb_data[:,:,3]
    confidence = s3_rgb_data[:,:,4]
    exception = s3_rgb_data[:,:,5]

    date_format = '%Y%m%dT%H%m%S'
    
    output_name = '-'.join([k.replace(' ', '-').upper(), 
                            output_startdate.strftime(date_format), 
                            output_startdate.strftime(date_format)])
    
    s3_rgb_composite(red, 
                     green,
                     blue, 
                     cloud, 
                     confidence,
                     exception,
                     geo_transform,
                     projection_ref, 
                     output_name + '.tif',
                     v['hfact'])
    
     # PNG
    gdal.Translate('{}.png'.format(output_name), 
                   '{}.tif'.format(output_name), 
                   format='PNG')

    os.remove('{}.png.aux.xml'.format(output_name))
    
    
    date_format = '%Y-%m-%dT%H:%m:%S'
    
    with open(output_name + '.tif.properties', 'w') as file:
        file.write('title={} ({}/{})\n'.format(k, 
                                               output_startdate.strftime(date_format),
                                               output_stopdate.strftime(date_format)))

        file.write('date={}Z/{}Z\n'.format(output_startdate.strftime(date_format),
                                               output_stopdate.strftime(date_format)))   
        file.write('geometry={}'.format(shape(item.geometry).wkt))

   

    with open(output_name + '.png.properties', 'w') as file:
        file.write('title={} - Quicklook ({}/{})\n'.format(k, 
                                               output_startdate.strftime(date_format),
                                               output_stopdate.strftime(date_format)))

        file.write('date={}Z/{}Z\n'.format(output_startdate.strftime(date_format),
                                           output_stopdate.strftime(date_format)))   
        file.write('geometry={}'.format(shape(item.geometry).wkt))

['s3_slstr.data/S8_BT_in.img', 's3_slstr.data/S1_reflectance_an.img', 's3_slstr.data/S5_reflectance_an.img', 's3_slstr.data/cloud_an.img', 's3_slstr.data/confidence_an.img', 's3_slstr.data/S2_exception_an.img']
2020-06-10 23:39:46 (INFO) ContrastEnhancement: The application has been launched with the following parameters :
- number of bins : 256
- contrast limtaition factor : 3
- spatial parameters : local with a thumbnail of 500 X 500
- equalisation of the luminance
- Min/Max parameters : automatic
2020-06-10 23:39:46 (INFO): Default RAM limit for OTB is 256 MB
2020-06-10 23:39:46 (INFO): GDAL maximum cache size is 1600 MB
2020-06-10 23:39:46 (INFO): OTB will use at most 2 threads
2020-06-10 23:39:46 (INFO): Estimated memory for full processing: 358.238MB (avail.: 256 MB), optimal image partitioning: 2 blocks
2020-06-10 23:39:46 (INFO): Estimation will be performed in 4 blocks of 1920x1920 pixels
Computing statistics: 100% [**************************************************] (3s)
2020

### Clean-up

In [ ]:
shutil.rmtree('{}.data'.format(write['file']))

In [ ]:
os.remove('{}.dim'.format(write['file']))

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.